In [ ]:
from google import genai
import os
from dotenv import load_dotenv

load_dotenv()

client = genai.Client(api_key=os.environ.get('GEMINI_API_KEY')) 

prompt = '''
You are an expert at extracting and structuring data from images of restaurant menus. I will provide you with images of a menu, and you must return the data in a structured JSON format.

Here are the specific details and constraints for the JSON output:

- **Restaurant Details:**
  - `restaurant_name`: Extract the name of the restaurant. If not explicitly mentioned, leave it.
  - `area_id`: A placeholder for an area ID (e.g., "123"). If not explicitly mentioned, leave it.
  - `area_name`: A placeholder for an area name (e.g., "Central City"). If not explicitly mentioned, leave it.

- **Menu Categories:**
  - `categories`: An array of objects. Each object should represent a menu category.
    - `id`: A unique, numerical ID for each category (e.g., 1, 2, 3).
    - `name`: The name of the category (e.g., "Special Calzone Menu", "Bao", "Dessert").
    - `image_url`: A placeholder for an image URL.
    - `availability`: A boolean value (`true` or `false`). Assume all items are available unless specified otherwise.
    - `rank`: An integer to determine the display order. Use 1, 2, 3, etc. based on the order in the image.

- **Menu Items:**
  - `items`: An array of objects within each category. Each object should represent a menu item.
    - `name`: The name of the dish (e.g., "Three Cheese Caprese").
    - `description`: The description of the dish, including ingredients (e.g., "Mozzarella+Cheddar+Cream Cheese +Tomato+Basil+Balsamic Drizzle").
    - `price`: The numerical price of the item.
    - `rank`: An integer to determine the display order within the category.
    - `image_url`: A placeholder for an image URL.
    - `stock_status`: A string. Assume "In Stock" unless a clear indication of being out of stock is present (e.g., a "sold out" icon).

- **Customizations:**
  - `customizations`: An array of objects. This should only be used for items with add-ons or variations.
    - `group_id`: A unique ID for the customization group.
    - `group_name`: The name of the group (e.g., "Add On").
    - `min_selection`: The minimum number of selections allowed.
    - `max_selection`: The maximum number of selections allowed.
    - `variations`: An array of objects for each customization option.
      - `name`: The name of the variation (e.g., "Gochujang Chicken").
      - `price`: The price of the variation.

**Important Instructions:**
- Carefully analyze the menu images to extract all relevant data.
- Ensure the JSON is properly formatted with correct syntax (commas, brackets, etc.).
- Use placeholders for `image_url`, `restaurant_name`, `area_id`, and `area_name` as you will not be able to generate these from the image.
- Pay close attention to items with multiple prices or add-ons and structure them correctly. For items like the "Mexican Style" Calzone, which has two prices, create two separate item entries. The first entry should have the first price, and the second should have the second price with a note in the description (e.g., "Paneer or Chicken"). Similarly, for the "Korean Garlic Buns," create a customization group for the "Add On" options.

Return only the final JSON object. Do not include any additional text, explanations, or conversational fillers in your response.'''

results = []
for i in range(2):
    uploaded_file = client.files.upload(file = f'data\\task_menu_{i+1}.png')
    # uploaded_file = client.files.upload(file = os.path.join(os.getcwd(),f'data/task_menu_{i+1}.png'))
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents = [prompt,uploaded_file]
    )
    results.append(response.text)
    

In [ ]:
import json
print(json.loads(results[0]))

In [ ]:

print(results[0])


In [ ]:
raw_str = results[0]

# remove the first line and last line if they are ```json and ```
lines = raw_str.splitlines()
if lines[0].strip().startswith("```"):
    lines = lines[1:]
if lines[-1].strip().startswith("```"):
    lines = lines[:-1]

clean_str = "\n".join(lines)
print(clean_str)

In [ ]:
print(json.loads(clean_str))

In [ ]:
print(response)

In [ ]:
import os
import json
FilePath = os.path.join(os.getcwd(),'data\\data_reference.json')
# FilePath = D:\pikky\menu-organizer\data\data_reference.json')
try:
    with open(FilePath, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    # Re-shape into the same structure extractImage() would return
    # so flatten_menu() works without changes
    print((raw_data))

except Exception as e:
    print(f"[extract_menu_json] Failed: {e}")

In [ ]:
import time

FilePath = 'data\\task_menu_1.png'
def extractImage( category_name,category_id ,max_retries = 3):
    
    prompt = f'''
        You are an expert at extracting and structuring data from images of restaurant menus.

        TASK
        - From the provided menu images, extract ONLY the items that belong to the category named: "{category_name}".
        - If the menu uses a near-synonym or plural/singular variant of the category (e.g., "Burger" vs "Burgers"), treat it as the same category.
        - Do NOT invent items. If none match, return an empty items array.

        OUTPUT FORMAT
        Return a single JSON object with this shape, with the item_category_id have shared (and no extra text):

        {{
        "items": [
            {{
            "itemid": "",                       // leave empty for new items
            "itemallowvariation": "0"|"1",      // "1" if variations are present, else "0"
            "itemname": "<string>",
            "itemrank": "<integer as string>",  // 1-based order; read top-to-bottom, left-to-right within the category
            "item_categoryid": "{category_id}", // leave "" if unknown
            "price": "<numeric as string>",     // e.g., "140"; strip currency symbols and punctuation
            "active": "1",                      // default "1"
            "item_favorite": "0",               // default "0"
            "itemallowaddon": "0"|"1",          // "1" if add-ons are present, else "0"
            "itemaddonbasedon": "0",            // default "0" (add-ons not tied to variation)
            "instock": "2",                     // default "2" (in stock); use "0" if clearly sold out
            "ignore_taxes": "0",                // default "0"
            "ignore_discounts": "0",            // default "0"
            "days": "-1",                       // default "-1"
            "item_attributeid": "",             // unknown => empty string
            "itemdescription": "<string>",      // concise description; join bullet points with ", "
            "minimumpreparationtime": "",       // unknown => empty string
            "item_image_url": "",               // placeholder
            "variation": [
                // present only when itemallowvariation == "1"
                {{ "name": "<string>", "price": "<numeric as string>" }}
            ],
            "addon": [
                // present only when itemallowaddon == "1"
                {{
                "addon_group_id": "",                 // leave empty if unknown
                "addon_item_selection": "S"|"M",      // "S" = single choice; "M" = multiple allowed
                "addon_item_selection_min": "<int as string>",
                "addon_item_selection_max": "<int as string>"
                }}
            ],
            "item_tax": ""                     // do not invent tax IDs; leave empty if unknown
            }}
        ]
        }}

        EXTRACTION RULES
        1) CATEGORY SCOPING
        - Only include items visually under the heading that matches "{category_name}" or a close lexical variant.
        - If headings are ambiguous, prefer the nearest visible header above the items.

        2) PRICE PARSING
        - Normalize prices to numbers-as-strings (e.g., "₹140/-" → "140").
        - If an item shows multiple sizes/variants with different prices (e.g., "Half 140 / Full 220"):
            • Prefer "variation" entries with name, price and set "itemallowvariation" = "1".
            • If the menu clearly lists the variants as separate named items, create separate items instead.
        - If both a base price and variations exist, set the base price in "price" and also include variations.

        3) RANKING
        - "itemrank" starts at "1" and increments by reading order within the category (top→bottom, left→right).

        4) ADD-ONS / CUSTOMIZATIONS
        - When the menu offers optional extras (e.g., “Add Cheese +30”, “Choose any 2 sauces”):
            • Set "itemallowaddon" = "1".
            • Use "addon_item_selection" = "S" if the text implies “choose 1”; use "M" if “choose any”/“choose up to N”.
            • Set min/max from the text; if unspecified, use min "0" and max "1" for S, or a reasonable observed max for M.
            • Do not invent "addon_group_id"; leave it empty if unknown.
        - Do not put add-ons into "variation". Variations are mutually exclusive forms of the item; add-ons are optional extras.

        5) STOCK
        - Default "instock" = "2".
        - If the item or its label clearly indicates unavailability (e.g., “Sold Out”, crossed-out), set "instock" = "0" and keep "active" = "1".

        6) TEXT CLEANUP
        - Preserve exact item names when possible; remove obvious OCR artifacts.
        - "itemdescription": concise, readable sentence/phrase. Include notable ingredients or style cues from the image.

        7) SAFETY & HALLUCINATION
        - Do NOT guess tax IDs, attribute IDs, prep time, or group IDs; leave those fields empty if not visible.
        - Do NOT add categories or items not present in the image.
        - Do NOT add decorative sections, marketing blurbs, or explanations (e.g., “What’s a Calzone”). Only extract actual menu items under category headings.

        RETURN FORMAT
        - Return only the final JSON object as specified above. No prose, no Markdown, no explanations.
    '''

    uploaded_file = client.files.upload(file = FilePath)
    attempt = 0
    while attempt < max_retries:
        try:
            uploaded_file = client.files.upload(file=FilePath)
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=[prompt, uploaded_file]
            )
            
            # lines = response.text.splitlines()
            lines = response.text
            # Strip triple backticks if present
            if lines and lines[0].strip().startswith("```"):
                lines = lines[1:]
            if lines and lines[-1].strip().startswith("```"):
                lines = lines[:-1]
            return lines

        except Exception as e:
            wait_time = 2 ** attempt  # exponential backoff: 1, 2, 4...
            print(f"[extractImage] Attempt {attempt+1} failed: {e}. Retrying in {wait_time}s...")
            time.sleep(wait_time)
            attempt += 1

    print("[extractImage] Max retries reached, giving up.")

    return ""

In [ ]:
from google import genai
client = genai.Client(api_key=os.environ.get('GEMINI_API_KEY')) 

category_id = "4796268"
category_name = "Bao"
new_items = extractImage(category_name=category_name,category_id=category_id)
clean = new_items.strip().removeprefix("```json").removesuffix("```").strip()
# print(clean)
json.loads(clean)
raw_data["items"].append(json.loads(new_items))

In [ ]:
# print(type(clean))
# clean = json.loads(clean)
print(type(clean["items"]))

In [ ]:
print(new_items)